<a href="https://colab.research.google.com/github/AnikethDandu/traffic-sign-classification/blob/main/TrafficSignClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolutional Neural Network class

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
  def __init__(self):
    # Input image: 32x32x3
    self.PADDING_SIZE = 1
    self.KERNEL_SIZE = 3
    self.STRIDE = 1
    self.POOL_SIZE = 2
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv2 = nn.Conv2d(32, 64, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv3 = nn.Conv2d(64, 128, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv4 = nn.Conv2d(128, 256, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.fc1 = nn.Linear(2304, 512)
    self.fc2 = nn.Linear(512, 43)
    
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv2(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv3(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv4(x)), self.POOL_SIZE)
    x = x.flatten(start_dim=1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


    

In [16]:
import pandas as pd
import os
import cv2

from torch.utils.data import Dataset
class TrafficSignDataset(Dataset):
  def __init__(self, train, root_dir, img_size):
    self.train = train
    self.root_dir = root_dir
    self.df = pd.read_csv(os.path.join(root_dir, 'Train.csv' if train else 'Test.csv'))
    self.img_size = img_size

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    image = cv2.imread(os.path.join(self.root_dir, self.df.iloc[idx][7]), cv2.IMREAD_COLOR)
    image = cv2.resize(image, (self.img_size, self.img_size))
    label = self.df.iloc[idx][6]
    sample = {'image': torch.tensor(image), 'label': self.df.iloc[idx][6]}
    return sample


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

EPOCHS = 10
BATCH_SIZE = 32

training_data = []
testing_data = []

image_size = (-1, 3, 50, 50)

training_dataset = TrafficSignDataset(train=True, root_dir='traffic_sign_images', img_size=50)
testing_dataset = TrafficSignDataset(train=False, root_dir='traffic_sign_images', img_size=50)

train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_dataloader = DataLoader(testing_dataset, batch_size=1, shuffle=True)

total_images = 0
class_img_count = []
for folder in os.listdir('traffic_sign_images/Train'):
  if folder != '.DS_Store':
    image_count = len([img for img in os.listdir(os.path.join('traffic_sign_images/Train', folder))])
    class_img_count.append(image_count)
    total_images += image_count

FINAL_WEIGHTS = torch.Tensor([1 - img_count/total_images for img_count in class_img_count]).to(device)
CNN = ConvNet().to(device)
optimizer = optim.Adam(CNN.parameters(), lr=0.0015)
criterion = nn.CrossEntropyLoss(weight=FINAL_WEIGHTS)

for epoch in range(EPOCHS):
  for batch_idx, batch in enumerate(train_dataloader):
    batch_imgs, batch_lbls = batch["image"].view(image_size) / 255.0, batch["label"]
    batch_labels = [0 for i in range(32)]
    for label_idx, label in enumerate(batch_lbls):
      batch_labels[label_idx] = label.item()

    batch_imgs = batch_imgs
    batch_labels = batch_labels
    
    optimizer.zero_grad()
    outputs = CNN(batch_imgs.to(device))
    loss = criterion(outputs, torch.tensor([label for label in batch_labels], device=device).long())
    loss.backward()
    optimizer.step()
  print(f'Epoch: {epoch + 1}, Loss: {loss}')

with torch.no_grad():
  total_classes = {}
  class_correct = {}
  total_images = 0
  total_correct = 0

  for batch_idx, batch in enumerate(test_dataloader):
    test_image, test_label = batch['image'].view(image_size) / 255.0, batch['label'].item()
    correct_class = test_label
    test_image = test_image.to(device)
    predicted_class = torch.argmax(CNN(test_image)[0])
    
    total_images += 1
    total_classes[predicted_class.item()] = total_classes[predicted_class.item()] + 1 if predicted_class.item() in total_classes else 1
    
    if predicted_class == correct_class:
      total_correct += 1
      class_correct[correct_class] = class_correct[correct_class] + 1 if correct_class in class_correct else 1
print([f'Accuracy for {img_class}: {round(100 * class_correct[img_class] / total_classes[img_class], 3)}%' for img_class in class_correct])
print(f'Raw class score: {class_correct}')
print(f'Total images correct: {total_correct}, Total images: {total_images}, Total accuracy: {round(100 * total_correct / total_images, 3)}%')



Epoch: 1, Loss: 0.8086117506027222
Epoch: 2, Loss: 0.248038649559021
Epoch: 3, Loss: 0.2852267622947693
Epoch: 4, Loss: 0.3402959406375885
Epoch: 5, Loss: 0.15202629566192627
Epoch: 6, Loss: 0.054856497794389725
Epoch: 7, Loss: 0.21575146913528442
Epoch: 8, Loss: 0.06401488929986954
Epoch: 9, Loss: 0.045847516506910324
Epoch: 10, Loss: 0.13893479108810425
['Accuracy for 33: 87.168%', 'Accuracy for 9: 76.599%', 'Accuracy for 12: 96.679%', 'Accuracy for 10: 99.278%', 'Accuracy for 4: 77.715%', 'Accuracy for 13: 95.174%', 'Accuracy for 8: 75.714%', 'Accuracy for 19: 36.129%', 'Accuracy for 38: 95.559%', 'Accuracy for 25: 81.619%', 'Accuracy for 14: 85.567%', 'Accuracy for 1: 85.118%', 'Accuracy for 42: 96.923%', 'Accuracy for 27: 41.176%', 'Accuracy for 2: 74.565%', 'Accuracy for 35: 95.798%', 'Accuracy for 7: 80.916%', 'Accuracy for 5: 77.969%', 'Accuracy for 31: 82.06%', 'Accuracy for 18: 62.613%', 'Accuracy for 29: 42.574%', 'Accuracy for 23: 69.231%', 'Accuracy for 3: 63.636%', 'Accur

In [ ]:
'''
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/gdrive/My\ Drive/ColabNotebooks/Data/ traffic_sign_images.zip
!unzip traffic_sign_images.zip/traffic_sign_images.zip
!rm -r traffic_sign_images.zip/
'''